In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle, wordvectors_centroid
from somhos.config.paths import *

Default path

In [2]:
prefix_path = "../../"
data_path = get_relative_path(prefix_path, V9GAMMA_PATH)
os.path.exists(data_path)

True

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (5000, 5000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 10000


In [5]:
from itertools import product
import numpy as np
from scipy.spatial.distance import cosine as cosine_distance

Load counts documents by word

In [6]:
docs_samples_word_doc_count_path = get_relative_path(data_path, DOCS_SAMPLES_WORD_DOC_COUNT)
samples_word_in_docs_count = load_pickle(docs_samples_word_doc_count_path)

In [7]:
n_docs = len(docs_sample_a) + len(docs_sample_b)
t_idf = {t: np.log(n_docs/v) for t, v in samples_word_in_docs_count.items()}

In [8]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_content_for(index, samples_content, field='tokens'):
    return samples_content[index][field]

def tf_doc(terms):
    tf = Counter(terms)
    cnt = sum(tf.values())
    return {k: v/cnt for k, v in tf.items()}

m_measure_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    # Term frequencies by document
    tf_a = tf_doc(get_content_for(docs_sample_a[i], samples_content))
    tf_b = tf_doc(get_content_for(docs_sample_b[j], samples_content))
    # Term list
    t_list = list(set(tf_a) | set(tf_b))
    # TF-IDF
    tf_idf_a = np.zeros([len(t_list),])
    tf_idf_b = np.zeros([len(t_list),])
    for pos, t in enumerate(t_list):
        if t in tf_a:
            tf_idf_a[pos] = tf_a[t] * t_idf[t]
        if t in tf_b:
            tf_idf_b[pos] = tf_b[t] * t_idf[t]
    # Measure
    measure_sim = 1.0 - cosine_distance(tf_idf_a, tf_idf_b)
    m_measure_sim[i, j] = measure_sim

print(m_measure_sim)

[[0.01075387 0.00206247 0.00420691 ... 0.00566958 0.00147329 0.0079679 ]
 [0.00313217 0.01103955 0.01155472 ... 0.01543637 0.00282845 0.00718103]
 [0.00373985 0.         0.00749853 ... 0.00558708 0.00154656 0.02780748]
 ...
 [0.00126982 0.02567926 0.00055769 ... 0.00533172 0.00415633 0.00597354]
 [0.00616515 0.00312923 0.00174796 ... 0.00822055 0.00276756 0.02864621]
 [0.00437217 0.00440589 0.00088447 ... 0.01827149 0.00584807 0.00173718]]


In [9]:
docs_samples_measure_sim_path = get_relative_path(data_path, DOCS_SAMPLES_TF_IDF)
save_pickle(m_measure_sim, docs_samples_measure_sim_path)

In [10]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_measure_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_measure_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_measure_sim)

[[0.00765054 0.00453286 0.00640747 ... 0.01133526 0.00653855 0.00849204]
 [0.00719062 0.00426036 0.00602228 ... 0.01065382 0.00614548 0.00798153]
 [0.00608192 0.00360347 0.00509372 ... 0.00901114 0.00519793 0.00675088]
 ...
 [0.00965801 0.00572227 0.00808876 ... 0.01430958 0.00825424 0.01072031]
 [0.00893104 0.00529154 0.00747991 ... 0.01323248 0.00763293 0.00991338]
 [0.01137832 0.00674153 0.00952955 ... 0.01685842 0.0097245  0.01262983]]


In [11]:
docs_samples_measure_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_TF_IDF_UDV)
save_pickle(m_udv_measure_sim, docs_samples_measure_sim_udv_path)